# Topic-specific term associations through word representations
## How do Democrats and Republicans talk different about jobs

https://github.com/JasonKessler/scattertext

Cite as:
Jason S. Kessler. Scattertext: a Browser-Based Tool for Visualizing how Corpora Differ. Proceedings of the 54th Annual Meeting of the Association for Computational Linguistics (ACL): System Demonstrations. 2017.

Link to preprint: https://arxiv.org/abs/1703.00565

`
@article{kessler2017scattertext,
  author    = {Kessler, Jason S.},
  title     = {Scattertext: a Browser-Based Tool for Visualizing how Corpora Differ},
  booktitle = {ACL System Demonstrations},
  year      = {2017},
}
`

In [4]:
%matplotlib inline
import scattertext as st
from gensim.models import word2vec
import re, io, itertools
from pprint import pprint
import pandas as pd
import numpy as np
import spacy
import os, pkgutil, json, urllib
from urllib.request import urlopen
from IPython.display import IFrame
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:98% !important; }</style>"))

In [5]:
# nlp = spacy.load('en')
# If this doesn't work, please uncomment the following line and use a regex-based parser instead
nlp = st.whitespace_nlp_with_sentences

## Load the 2012 Conventions Dataset
### We'll limit the study to unigrams

In [6]:
convention_df = st.SampleCorpora.ConventionData2012.get_data()
convention_df['parsed'] = convention_df.text.apply(nlp)
corpus = (st.CorpusFromParsedDocuments(convention_df, category_col='party', parsed_col='parsed')
          .build()
          .get_unigram_corpus())

## Use Gensim to run Word2Vec on the corpus.
### Word2Vec encodes each word in a dense K-dimensional vector space
### Cosine distances between terms vectors correspond to semantic similarity 

In [7]:
# model = word2vec.Word2Vec(size=100, window=5, min_count=10, workers=4)
model = word2vec.Word2Vec(vector_size=100, window=5, min_count=10, workers=4)
model = st.Word2VecFromParsedCorpus(corpus, model).train(epochs=10000)
model.wv.most_similar('jobs')

[('businesses', 0.8398184776306152),
 ('create', 0.8260375261306763),
 ('spending', 0.795708417892456),
 ('energy', 0.7936553955078125),
 ('debt', 0.765021562576294),
 ('created', 0.7505785226821899),
 ('creating', 0.7439010739326477),
 ('taxes', 0.7438827753067017),
 ('tax', 0.7356417179107666),
 ('000', 0.7315483689308167)]

In [8]:
corpus._df[corpus._parsed_col].apply(lambda x: len(list(x.sents))).sum()
#model.corpus_count

9125

## Draw the Scattertext by only coloring points that have are associated with a category (p < 0.05 via log-odds w/ prior)
### The top Democratic and Republican terms are raked by their similarity to "jobs"
#### Only the terms associated to a category are considered. 
### On the far right, the most similar terms, regardless of category association, are listed.

In [9]:
target_term = 'jobs'

html = st.word_similarity_explorer_gensim(corpus,
                                          category='democrat',
                                          category_name='Democratic',
                                          not_category_name='Republican',
                                          target_term=target_term,
                                          minimum_term_frequency=5,
                                          width_in_pixels=1000,
                                          word2vec=model,
                                          metadata=convention_df['speaker'])
file_name = 'output/demo_similarity_gensim.html'
open(file_name, 'wb').write(html.encode('utf-8'))
IFrame(src=file_name, width = 1200, height=700)

C:\ProgramData\Anaconda3\lib\site-packages\scattertext\Scalers.py:212: RuntimeWarning: invalid value encountered in true_divide
  vec_ss = (vec_ss - vec_ss.min()) * 1. / (vec_ss.max() - vec_ss.min())


## Instead of using vectors trained on the Corpus, we can use the spaCy-provided word vectors trained on the Common Crawl Corpus.
### These are trained on a lot more data, but aren't specific to the corpus

In [11]:
# Note: this will fail if you did not use spaCy as your parser.
html = st.word_similarity_explorer(corpus,
                                   category='democrat',
                                   category_name='Democratic',
                                   not_category_name='Republican',
                                   target_term='jobs',
                                   minimum_term_frequency=5,
                                   width_in_pixels=1000,
                                   metadata=convention_df['speaker'])
file_name = 'output/demo_similarity.html'
open(file_name, 'wb').write(html.encode('utf-8'))
IFrame(src=file_name, width = 1200, height=700)